# HW8-ASSN1: Which community districts in NYC show the highest number of complaints?

In [45]:
import os
import pandas as pd
import numpy as np
import re
import sys
PUIdata = os.getenv('PUIDATA')

In [46]:
comdistdata = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,17,39,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
comdistdata.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11224,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)"
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,...,NaN,NaN,NaN,1-1-1173130914,NaN,NaN,NaN,NaN,NaN,NaN
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,Street,11234,1157 EAST 57 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)"
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11218,722 EAST 4 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)"
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NYC Street Address,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
comdistdata.drop(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough',
       'School Name', 'School Number', 'School Region', 'School Code',
       'School Phone Number', 'School Address', 'School City', 'School State',
       'School Zip', 'School Not Found', 'School or Citywide Complaint',
       'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location',
       'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp',
       'Bridge Highway Segment', 'Garage Lot Name', 'Ferry Direction',
       'Ferry Terminal Name', 'Location'], axis=1, inplace=True)

In [49]:
comdistdata.head()

,Complaint Type,Descriptor,Community Board,Borough,Latitude,Longitude
0,Street Condition,Pothole,13 BROOKLYN,BROOKLYN,40.573431,-73.991742
1,Opinion for the Mayor,HOUSING,0 Unspecified,Unspecified,NaN,NaN
2,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,18 BROOKLYN,BROOKLYN,40.625004,-73.920726
3,Illegal Parking,Blocked Hydrant,12 BROOKLYN,BROOKLYN,40.634522,-73.974790
4,Benefit Card Replacement,Medicaid,0 Unspecified,Unspecified,NaN,NaN


In [50]:
len(comdistdata['Community Board'].unique())

77

In [51]:
comdistdata['Community Board'].unique()

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

# All B/C represented? There are 77 unique values, of which 6 are "Unspecified" (one for each of the boroughs and one general).  Why so many?

In [52]:
len(comdistdata)

10187766

In [53]:
comdistdata.drop_duplicates().shape

(4463875, 6)

# After dropping duplicates, we are left with a significantly smaller data set.  This is plausible, since this would account for unique callers calling multiple times.

In [54]:
len(comdistdata['Community Board'].unique())

77

In [55]:
comdistdata['Community Board'].unique()

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

In [56]:
comdistdata.isnull().sum()

Complaint Type          0
Descriptor          31057
Community Board         0
Borough                 0
Latitude           878752
Longitude          878752
dtype: int64

# There are missing values present in the Descriptor, Latitude, and Longitude fields.  It is reasonable that Complaint Type, CD, and Borough are all consistently filled out, as these constitute the basic indicators for any call.  Descriptor requires classification and Lat/Long require pinpointing the specific call.

# Implausible community districts include any including the word "Unspecified" as well as any with number greater than 18 (Brooklyn has the most CDs, with 18).

In [57]:
def comdistflag(data):
    '''
    This function performs a search that returns a value of 0 if either "Unspecified" or a number > 18 appear in the data.
    Otherwise, it returns a value of 1.
    '''
    if re.search(r"Unspecified", data):
        return 0
    elif int(re.search(r"(\d\d*)\s\w+", data).group(1))>18:
        return 0
    else:
        return 1

In [58]:
mask = comdistdata['Community Board'].apply(comdistflag)

In [59]:
comdistdata["masks"] = mask

In [60]:
comdistdata_red = comdistdata[comdistdata.masks==1]

In [61]:
comdistdata_red['Community Board'].unique().shape

(59,)

# The function successfully reduced the number of CDs down to 59 (the right total).

In [62]:
cdd_r_sorted = comdistdata_red['Complaint Type'].groupby(comdistdata_red['Community Board']).count()

In [63]:
cdd_r_sorted = pd.DataFrame(cdd_r_sorted)

In [64]:
cddsorted = cdd_r_sorted.reset_index().sort(columns=['Complaint Type'], ascending=False).reset_index(drop=True)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [65]:
cddsorted = cddsorted.rename(columns = {'Complaint Type':'counts'})

In [66]:
cddsorted.head()

,Community Board,counts
0,12 MANHATTAN,268053
1,12 QUEENS,229383
2,03 BROOKLYN,197306
3,07 QUEENS,195961
4,17 BROOKLYN,191720


# This table shows the rankings of CDs based on the total count of 311 calls.  Without further analysis into the complaint type, however, we cannot draw any meaningful conclusions other than by quantity.

In [67]:
df_demog = pd.read_csv(PUIdata + "/Final_Demographics.csv")

In [68]:
df_demog.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [69]:
df_demog['cd_id'].unique().shape

(59,)

# There are 59 CD's in the demographics data.

In [70]:
df_demog.columns

Index(['FIPS', 'cd_id', 'Total Population',
       'Population Density (per sq. mile)', '% Total Population: Male',
       '% Total Population: 18 to 24 Years',
       '% Total Population: 25 to 34 Years',
       '% Total Population: 35 to 44 Years',
       '% Population 5 Years And Over: Speak Only English',
       '% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       '% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       '% Employed Civilian Population 16 Years And Over: Service occupations',
       '% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       '% Employed Civ

In [71]:
density = df_demog[['cd_id', 'Population Density (per sq. mile)']]

In [72]:
density.head()

,cd_id,Population Density (per sq. mile)
0,BX08,31229.95006
1,BX12,19966.67839
2,BX10,12913.81703
3,BX11,35677.95453
4,BX03,39405.79222


# I focused on the population density by CD.  We can cross-reference the population density (by CD) with the number of complaint calls - it would make sense that we receive more complaint calls in districts with a denser population.

In [73]:
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [74]:
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [75]:
df_infr['Qualifying Name'].unique().shape

(55,)

# There are 55 CD's in the infrastructure data.

In [76]:
df_infr.columns

Index(['FIPS', 'Geographic Identifier', 'Qualifying Name', 'Households',
       'Households: With An Internet Subscription',
       'Households: Dial-Up Alone', 'Households: Dsl',
       'Households: With Mobile Broadband',
       'Households: Without Mobile Broadband', 'Households: Cable Modem',
       'Households: With Mobile Broadband.1',
       'Households: Without Mobile Broadband.1', 'Households: Fiber-Optic',
       'Households: With Mobile Broadband.2',
       'Households: Without Mobile Broadband.2',
       'Households: Satellite Internet Service',
       'Households: With Mobile Broadband.3',
       'Households: Without Mobile Broadband.3',
       'Households: Two or More Fixed Broadband Types, or Other',
       'Households: With Mobile Broadband.4',
       'Households: Without Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup',
       'Households: Internet Access Without A Subscription',
       'Households: No Internet Access',
       '% Househol

In [77]:
mobilebb = df_infr.iloc[:,[2, 7, 10, 13, 16, 19, 21]]

In [78]:
mobilebb['Mobile Broadband'] = mobilebb.sum(axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [79]:
nomobilebb = df_infr.iloc[:,[2, 8, 11, 14, 17, 20, 22]]

In [80]:
nomobilebb['Without Mobile Broadband'] = nomobilebb.sum(axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [81]:
mobilebb_info = pd.concat([mobilebb, nomobilebb], axis=1)

In [82]:
mobilebb_info = mobilebb_info.iloc[:,[0, 7, 15]]

In [83]:
mobilebb_info.head()

,Qualifying Name,Mobile Broadband,Without Mobile Broadband
0,"NYC-Bronx Community District 8--Riverdale, New...",17527,15943
1,"NYC-Bronx Community District 12--Wakefield, Ne...",11414,22542
2,"NYC-Bronx Community District 10--Co-op City, N...",12064,23389
3,NYC-Bronx Community District 11--Pelham Parkwa...,11749,22835
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",16292,22435


In [84]:
high_speed =  df_infr.iloc[:,[2, 6, 9, 12]]
high_speed['High'] = high_speed.sum(axis=1)
low_speed = df_infr.iloc[:, [2, 5, 15]]
low_speed['Low'] = low_speed.sum(axis=1)
it_info = pd.concat([high_speed, low_speed], axis=1)
it_info = it_info.ix[:,[0, 4, 8]]

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [85]:
it_info.head()

,Qualifying Name,High,Low
0,"NYC-Bronx Community District 8--Riverdale, New...",22930,519
1,"NYC-Bronx Community District 12--Wakefield, Ne...",25718,309
2,"NYC-Bronx Community District 10--Co-op City, N...",25397,158
3,NYC-Bronx Community District 11--Pelham Parkwa...,24830,449
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",18654,282


# There should be a negative correlation between high broadband availability and complaint calls, as the 311 website allows for online submission of complaint requests.  We can cross-reference the CDs with the highest availability, and this should, in theory, be a reverse of the CDs by complaint calls.

In [86]:
def harmonizer_1(data):
    if re.search(r"Bronx", data, re.IGNORECASE):
        return re.sub(r"(\d\d)\s.+", r"BX\1", data)
    elif re.search(r"Queen", data, re.IGNORECASE):
        return re.sub(r"(\d\d)\s.+", r"QN\1", data)
    elif re.search(r"STATEN ISLAND", data, re.IGNORECASE):
        return re.sub(r"(\d\d)\s.+", r"SI\1", data)
    elif re.search(r"BROOKLYN", data, re.IGNORECASE):
        return re.sub(r"(\d\d)\s.+", r"BK\1", data)
    elif re.search(r"manhattan", data, re.IGNORECASE):
        return re.sub(r"(\d\d)\s.+", r"MN\1", data)

In [87]:
pat1 = re.compile(r"\w+-\w+\s*\w*\s\w+\s\w+\s(\d\d)\s*&*\d*.*", re.IGNORECASE)
pat2 = re.compile(r"\w+-\w+\s*\w*\s\w+\s\w+\s(\d)\s*&*\d*.*", re.IGNORECASE)

In [88]:
def harmonizer_2(data):
    if re.search(r"Bronx", data,re.IGNORECASE):
        if re.search(pat1, data):
            return re.sub(pat1, r"BX\1", data)
        else:
            return re.sub(pat2, r"BX0\1", data)
    elif re.search(r"Queen", data,re.IGNORECASE):
         if re.search(pat1, data):
            return re.sub(pat1, r"QN\1", data)
         else:
            return re.sub(pat2, r"QN0\1", data)
    elif re.search(r"STATEN ISLAND", data, re.IGNORECASE):
         if re.search(pat1, data, ):
            return re.sub(pat1, r"SI\1", data)
         else:
            return re.sub(pat2, r"SI0\1", data)
    elif re.search(r"BROOKLYN", data, re.IGNORECASE):
         if re.search(pat1, data):
            return re.sub(pat1, r"BK\1", data)
         else:
            return re.sub(pat2, r"BK0\1", data)
    elif re.search(r"manhattan", data, re.IGNORECASE):
         if re.search(pat1, data):
            return re.sub(pat1, r"MN\1", data)
         else:
            return re.sub(pat2, r"MN0\1", data)

In [89]:
demo = density.drop_duplicates(subset='Population Density (per sq. mile)')

In [90]:
demo = demo.rename(columns = {'cd_id':'district'})

In [91]:
demo["district"][demo["district"]=="MN05"] = "MN04"

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [92]:
first = mobilebb_info.merge(it_info, on="Qualifying Name")

In [93]:
first["district"] = first["Qualifying Name"].apply(harmonizer_2)

In [94]:
first.drop(["Qualifying Name"], axis=1, inplace=1)

In [95]:
first.drop_duplicates(inplace=1)

In [96]:
cddsorted["district"] = cddsorted["Community Board"].apply(harmonizer_1)

In [98]:
cddsorted.drop(["Community Board"], axis=1, inplace=1)

In [99]:
cddsorted = cddsorted.drop_duplicates(subset="counts")

In [101]:
final = demo.merge(first, on="district", how="inner").merge(cddsorted, on="district", how="inner")

In [102]:
final

,district,Population Density (per sq. mile),Mobile Broadband,Without Mobile Broadband,High,Low,counts
0,BX08,31229.950060,17527,15943,22930,519,92733
1,BX12,19966.678390,11414,22542,25718,309,168455
2,BX10,12913.817030,12064,23389,25397,158,95496
3,BX11,35677.954530,11749,22835,24830,449,122081
4,BX03,39405.792220,16292,22435,18654,282,75134
5,BX07,86487.077920,16709,19414,22626,0,185729
6,BX05,87974.348600,15857,13401,12431,415,161078
7,BX04,71270.882190,18342,14419,16053,732,181953
8,BX09,42752.506900,12892,34775,32042,669,147457
9,BX01,34412.075240,15981,19144,20357,1145,74631


# This is a combined data frame including district, median household income, mobile broadband availability, and counts of complaint calls.  With this, we can sort by multiple columns and test our hypotheses on the effects on complaint calls.  There should be a negative correlation between mobile availability and calls.  There should be a positive correlation between population density and calls.